This notebook goes through the process of building a ReAct agent from scratch. [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/synacktraa/ai-agents/blob/main/built-from-scratch/ReAct-agent.ipynb)

**Tutorial by James Murdza**: 
<a href="https://youtu.be/C0QdSBoJiMs">
    <img src="https://img.shields.io/badge/YouTube-FF0000?style=for-the-badge&logo=youtube&logoColor=white" alt="YouTube" width=90px height=20px>
</a>


In [ ]:
%%capture --no-stderr
%pip install "docstring-parser>=0.16" duckduckgo-search litellm loguru requests sympy

## Implementing `ReActTool`

First, we're going to implement a `ReActTool` class. 

It will be responsible for:

- Wrapping a function that must have exactly one parameter of a supported type (string, integer, float, boolean, list, dictionary, tuple, or set). If no type is specified for the parameter, it will be treated as a string. The function must also include a docstring that describes what it does.

- Converting raw string inputs into the correct type before passing them to the wrapped function. For example, if the wrapped function expects an integer, the ReActTool will automatically convert string inputs like "123" into the integer 123.

- Catches all exception raised by the associated function and returns a ExceptionInfo instead of just halting the execution. The exception info can be used by agent to understand what went wrong and how to fix it. This also helps in when defining the function, as we don't have to explicitly catch and handle exceptions.

In [3]:
import inspect
from dataclasses import dataclass
from typing import Any, Callable, Generic, Protocol, TypeVar

from docstring_parser import parse


T_Param = TypeVar("T_Param", str, int, float, bool, list, dict, tuple, set)
"""Type representing the accepted parameter types for the function."""

class SupportsStr(Protocol):
    def __str__(self) -> str:
        ...

T_Retval = TypeVar("T_Retval", bound=SupportsStr)
"""Type representing the return value of the function that can be converted to string."""


@dataclass
class ExceptionInfo:
    """Information about an exception."""
    name: str
    message: str


def validate_and_get_cast_type(__fn: Callable[..., Any]) -> type:
    """Validate if the function is compatible with ReAct agent and get the cast type."""

    if not inspect.isfunction(__fn):
        raise TypeError(f"Expected a function, got {type(__fn)}")
    
    fn_params = list(inspect.signature(__fn).parameters.values())
    if len(fn_params) != 1:
        raise ValueError(f"Expected a function with a single parameter, got {len(fn_params)}")
    
    cast_type = fn_params[0].annotation
    if cast_type in (Any, inspect._empty):
        return str  # Default to string if any or no type is specified
    
    if cast_type not in T_Param.__constraints__:
        constraints = tuple(map(lambda x: x.__name__, T_Param.__constraints__))
        raise ValueError(
            f"Expected a function with one of the following parameter types: {constraints}"
        )
    return cast_type
        

class ReActTool(Generic[T_Param, T_Retval]):
    """
    Class for creating a ReAct agent compatible tool from a function.
    
    A ReAct agent tool is supposed to be a function that takes a single parameter.
    """
    def __init__(self, __fn: Callable[[T_Param], T_Retval]):

        self._cast_type = validate_and_get_cast_type(__fn)
        
        docstring = parse(__fn.__doc__ or "")
        if docstring.description is None:
            raise ValueError("Function must include a description")
        
        self.__fn = __fn
        self.name = self.__fn.__name__
        self.description = docstring.description

    def __call__(self, __arg: T_Param) -> T_Retval:
        """Call the function associated with the adapter."""
        
        return self.__fn(__arg)
    
    def call_with_action_input(self, input: str) -> T_Retval:
        """
        Call the associated function with raw action input.
        Returns `ExceptionInfo` if any exception is raised.
        """

        try:
            return self.__fn(self._cast_type(input))
        except Exception as e:
            return ExceptionInfo(name=e.__class__.__name__, message=str(e))


def tool(__fn: Callable[[T_Param], T_Retval]) -> ReActTool[T_Param, T_Retval]:
    """Decorate a function to create a ReAct agent compatible tool."""
    return ReActTool(__fn)

## Implementing a ReAct agent

Before we start with the implementation, let's get a basic understanding of the `ReAct` agent.

`ReAct` stands for `Reason` and `Action`. It is a type of agent that processes the user's query in an iterative manner. The agent has access to a set of tools which can be used to solve the query or its sub-problems (when the agent breaks down complex queries during reasoning).

The agent processes queries through the following loop:

- The agent receives a query and thinks about how to solve it
- If a tool is needed, the agent specifies:
  - Action: The name of the tool to use
  - Action Input: The input to provide to that tool
- The specified tool is called and its result becomes an Observation
- The agent then either:
  - Provides a Final Answer if it has solved the query
  - Or continues thinking and using more tools until it reaches a solution
- The loop stops when either a Final Answer is provided or no valid action can be taken

In [4]:
# Setup Logger
import sys

from loguru import logger

logger.remove()

logger.add(sys.stdout, format="<level>{message}</level>")
logger = logger.opt(colors=True)

def highlight_role(role: str) -> str:
    return f"<bg #0f0707><fg #ffffff>[{role.upper()}]</fg #ffffff></bg #0f0707>"

In [5]:
import re
from string import Template

from litellm import completion as get_completion


TOOL_TEMPLATE = Template("${name}: ${description}")
"""Template for tool description."""


INSTRUCTION_TEMPLATE = Template("""${description}
You can only utilize the following tools to answer the user's query:
${tools_representation}

Use the following format:

Query: the input query you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of ${tool_names}
Action Input: the input to the action
Observation: the result of the action
... (the Thought/Action/Observation can repeat any number of times)
Thought: I now know the final answer!
Final Answer: the answer to the original input query\
""")
"""Instruction template for ReAct agent."""


class NoResult:
    """A sentinel value to indicate action or tool is not available."""


class ReActAgent:
    """
    React agent implementation that uses tools to solve given tasks.
    The agent can be initialized with a list of functions or ReActTool instances.

    This agent uses litellm to connect with Language Models.
    Make sure to set the api key of the provider you are using.

    Examples:
        ```python
        # Define your functions or tools
        from duckduckgo_search import DDGS, exceptions
        from sympy import sympify

        def search_text(text: str):
            \"\"\"Search for text in the web.\"\"\"
            
            try:
                return DDGS().text(keywords=text)
            except exceptions.RatelimitException:
                return "Rate limit exceeded. Return final answer."
        
        @tool   
        def calulate_math_expression_with_sympy(expression):
            \"\"\"Calculate a mathematical expression using sympy.\"\"\"

            try:
                result = sympify(expression)
                return result
            except Exception as e:
                return f"Error in calculation: {str(e)}"

        # Initialize the agent
        agent = ReactAgent(search_text, calulate_math_expression_with_sympy)

        # Call the agent
        agent(query="What is the meaning of life?", model="ollama/phi4")

        # See the final instruction
        agent.instruction

        # Take a peek at the history
        agent.show_history()
        ```
    """
    def __init__(
        self, 
        *functions_or_tools: Callable[[T_Param], T_Retval] | ReActTool[T_Param, T_Retval],
        default_model: str,
        description: str | None = None,
    ) -> None:
        """
        Args:
            functions_or_tools: A list of functions or ReActTool instances.
            default_model: The default model to use for the agent. Explore models: https://docs.litellm.ai/docs/providers
            description: A description that tells what an agent is and is supposed to do.
        """

        if not functions_or_tools:
            raise ValueError("Atleast one function or tool is required.")

        tools_representation = ""
        self._tool_registry: dict[str, ReActTool] = {}
        for entry in functions_or_tools:  # Update the registry with ReActTool instances
            tool = entry if isinstance(entry, ReActTool) else ReActTool(entry)
            self._tool_registry[tool.name] = tool
            tool_repr = TOOL_TEMPLATE.substitute(name=tool.name, description=tool.description)
            tools_representation += tool_repr + "\n"
        
        self.default_model = default_model
        description = description.strip() + "\n" if description else ""
        self.instruction = INSTRUCTION_TEMPLATE.substitute(
            description=description,
            tools_representation=tools_representation.strip(),
            tool_names=list(self._tool_registry)
        )
        """Instruction for the agent."""

        # Intialize message history with system instruction
        self.__message_history = [{"role": "system", "content": self.instruction}]

    def _parse_section(self, text: str, section: str) -> str | None:
        """Parse the section of the message."""
        _match = re.search(rf"{section}: (.*)", text)
        if _match:
            return _match.group(1).strip()

    def parse_action_and_get_result(self, ai_message: str) -> str | NoResult:
        """Parse action and input from AI message and call the corresponding tool."""

        action = self._parse_section(ai_message, "Action")
        action_input = self._parse_section(ai_message, "Action Input")
        if action and action_input:
            logger.info(
                f"⚙️ Action: {action!r} || ⌨️ Input: {action_input!r}"
            )
            if tool := self._tool_registry.get(action):
                return str(tool.call_with_action_input(action_input))
        
        return NoResult    

    def process_ai_message(self, content: str) -> bool:
        """Process an AI message and tell if the agent should continue solving the query."""

        logger.info(highlight_role("self"))
        if thought := self._parse_section(content, "Thought"):
            logger.info(f"💭 Thought: {thought}")

        # If final answer is there, we tell the agent to stop.
        if final := self._parse_section(content, "Final Answer"):
            logger.info(f"📌 Final Answer: {final}")
            self.__message_history.append({"role": "assistant", "content": final})
            return False
        
        # Parse ai message and get the action result
        action_result = self.parse_action_and_get_result(content)
        if action_result is NoResult:
            return False
        
        logger.info(f"🔎 Observation: {action_result}")
        
        # Add the ai message and observation to the message history
        self.__message_history.extend([
            {"role": "assistant", "content": content.strip()},
            {"role": "user", "content": f"Observation: {action_result}"}
        ])
        return True

    def __call__(self, *, query: str, model: str | None = None) -> str:
        """
        Call the agent.

        Args:
            query: The query to complete
            model: The model to use in the place of the default model.
        """
        query = query.strip()
        logger.info(f"🖋️ Query: {query}\n")

        # Add user query to message history
        self.__message_history.append({"role": "user", "content": f"Query: {query}"})

        while True:
            # Get completion response from the model
            ai_message = get_completion(
                model or self.default_model, self.__message_history, stop=["Observation:"]
            ).choices[0].message.content
            
            # Process the AI message
            if not self.process_ai_message(ai_message):
                break
            
            logger.info("")
        
    @property
    def history(self) -> list[dict[str, str]]:
        """Messasge history"""
        return self.__message_history

    def show_history(self) -> None:
        """Display the message history."""
        for message in self.__message_history:
            logger.info(f"{highlight_role(message['role'])} \n{message['content']}\n")

    def clear_history(self) -> None:
        """Clear the message history."""
        self.__message_history = [{"role": "system", "content": self.instruction}]

### Creating a ReAct agent

I am using Groq because it is super fast and has a good free usage limit. Get your own key from [here](https://console.groq.com/keys). If you want to use other models, you can select one from the [LiteLLM providers](https://docs.litellm.ai/docs/providers).

In [6]:
import os
from getpass import getpass

def set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"{var}: ")

set_env("GROQ_API_KEY")  # Replace with your LLM compatible environment key.

In [7]:
from duckduckgo_search import DDGS
from sympy import sympify

def search_text(text: str):
    """Search for text in the web."""
    
    return DDGS().text(keywords=text)

## Decorating is not really required, normal functions will work fine. 
## I am just trying to keep things flexible.
@tool
def calulate_math_expression_with_sympy(expression):
    """Calculate a mathematical expression using sympy."""

    return sympify(expression)

agent = ReActAgent(
    search_text, calulate_math_expression_with_sympy, default_model="groq/llama-3.3-70b-versatile"
)
print(agent.instruction)


You can only utilize the following tools to answer the user's query:
search_text: Search for text in the web.
calulate_math_expression_with_sympy: Calculate a mathematical expression using sympy.

Use the following format:

Query: the input query you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of ['search_text', 'calulate_math_expression_with_sympy']
Action Input: the input to the action
Observation: the result of the action
... (the Thought/Action/Observation can repeat any number of times)
Thought: I now know the final answer!
Final Answer: the answer to the original input query


In [8]:
agent(query="Find some information about smolagents library.")

🖋️ Query: Find some information about smolagents library.

[SELF]
💭 Thought: The user is asking for information about the smolagents library, so I should try to find some relevant information online.
⚙️ Action: 'search_text' || ⌨️ Input: 'smolagents library'
🔎 Observation: [{'title': 'GitHub - huggingface/smolagents: smolagents: a barebones library for ...', 'href': 'https://github.com/huggingface/smolagents', 'body': 'smolagents is a library that enables you to run powerful agents in a few lines of code. It offers: Simplicity: the logic for agents fits in ~thousand lines of code (see agents.py).We kept abstractions to their minimal shape above raw code! 🧑\u200d💻 First-class support for Code Agents, i.e. agents that write their actions in code (as opposed to "agents being used to write code").'}, {'title': 'Smolagents : Huggingface AI Agent Framework', 'href': 'https://smolagents.org/', 'body': 'smolagents is a minimalist AI agent library developed by the Hugging Face team. It allows d

In [9]:
agent(query="Calculate square root of 144.")

🖋️ Query: Calculate square root of 144.

[SELF]
💭 Thought: To calculate the square root of 144, I can use a mathematical function.
⚙️ Action: 'calulate_math_expression_with_sympy' || ⌨️ Input: 'sqrt(144)'
🔎 Observation: 12

[SELF]
💭 Thought: I now know the final answer!
📌 Final Answer: The square root of 144 is 12.


In [10]:
agent.show_history()

[SYSTEM] 

You can only utilize the following tools to answer the user's query:
search_text: Search for text in the web.
calulate_math_expression_with_sympy: Calculate a mathematical expression using sympy.

Use the following format:

Query: the input query you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of ['search_text', 'calulate_math_expression_with_sympy']
Action Input: the input to the action
Observation: the result of the action
... (the Thought/Action/Observation can repeat any number of times)
Thought: I now know the final answer!
Final Answer: the answer to the original input query

[USER] 
Query: Find some information about smolagents library.

[ASSISTANT] 
Thought: The user is asking for information about the smolagents library, so I should try to find some relevant information online.
Action: search_text
Action Input: smolagents library

[USER] 
Observation: [{'title': 'GitHub - huggingface/smolagents: smolagents:

### Creating a ReAct agent with custom description

In [20]:
import requests
from dataclasses import dataclass
from typing import Any

def fetch_json(url: str) -> dict[str, Any]:
    """Fetch JSON data from a given URL."""
    
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

@dataclass
class WeatherData:
    temperature: float
    wind_speed: float

    def __str__(self) -> str:
        return f"Current temperature: {self.temperature}°C, Current wind speed: {self.wind_speed} km/h"

def get_weather(city_name: str) -> WeatherData:
    """Get weather data for a given city."""

    json_response = fetch_json(
        f"https://geocoding-api.open-meteo.com/v1/search?name={city_name}"
    )

    if results := json_response.get("results", []):
        latitude, longitude = results[0]["latitude"], results[0]["longitude"]
    else:
        raise LookupError(f"City {city_name!r} not found.")
    
    current_weather = fetch_json(
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}&current_weather=true"
    ).get("current_weather", {})

    try:
        return WeatherData(
            temperature=current_weather["temperature"],
            wind_speed=current_weather["windspeed"],
        )
    except KeyError:
        raise LookupError("Weather data not available for city {city_name!r}.")

research_agent = ReActAgent(
    search_text, get_weather, 
    default_model="groq/llama-3.3-70b-versatile",
    description="""\
You are a research agent who makes use of tools to find information and answer user queries.
"""
)
print(research_agent.instruction)

You are a research agent who makes use of tools to find information and answer user queries.

You can only utilize the following tools to answer the user's query:
search_text: Search for text in the web.
get_weather: Get weather data for a given city.

Use the following format:

Query: the input query you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of ['search_text', 'get_weather']
Action Input: the input to the action
Observation: the result of the action
... (the Thought/Action/Observation can repeat any number of times)
Thought: I now know the final answer!
Final Answer: the answer to the original input query


In [21]:
research_agent(
    query="Based on the current weather of Kolkata, find the best place to visit."
)

🖋️ Query: Based on the current weather of Kolkata, find the best place to visit.

[SELF]
💭 Thought: To determine the best place to visit in Kolkata based on the current weather, I first need to find out what the current weather in Kolkata is.
⚙️ Action: 'get_weather' || ⌨️ Input: 'Kolkata'
🔎 Observation: Current temperature: 25.8°C, Current wind speed: 8.4 km/h

[SELF]
💭 Thought: With a current temperature of 25.8°C and a wind speed of 8.4 km/h, the weather in Kolkata is mild and pleasant. This is ideal for outdoor activities. Now, I need to find out the best places to visit in Kolkata that are suitable for this kind of weather.
⚙️ Action: 'search_text' || ⌨️ Input: 'best places to visit in Kolkata for outdoor activities'
🔎 Observation: [{'title': '17 Best Outdoor Activities In Kolkata: Unique Finds! - WanderOn', 'href': 'https://wanderon.in/blogs/outdoor-activities-in-kolkata', 'body': 'Explore the top 17 outdoor activities in Kolkata, from thrilling adventures to nature walks at the 

### How does this agent handles exceptions?

In [17]:
def search_web(query: str) -> str:
    """Search the web for the given query."""
    raise ConnectionError("No Internet connection.")

agent = ReActAgent(
    search_web, default_model="groq/llama-3.3-70b-versatile"
)

In [18]:
agent(query="Find me some good tutorials on how to get started with Python.")

🖋️ Query: Find me some good tutorials on how to get started with Python.

[SELF]
💭 Thought: To find good tutorials on getting started with Python, I should search the web for beginner-friendly resources.
⚙️ Action: 'search_web' || ⌨️ Input: 'Python tutorials for beginners'
🔎 Observation: ExceptionInfo(name='ConnectionError', message='No Internet connection.')

[SELF]
💭 Thought: It seems I don't have an internet connection, which is necessary to search the web for Python tutorials. I'll have to wait until the connection is restored to proceed.
📌 Final Answer: Sorry, I was unable to retrieve any information due to a connection error. Please try again when the internet connection is available.


In [19]:
agent.show_history()

[SYSTEM] 

You can only utilize the following tools to answer the user's query:
search_web: Search the web for the given query.

Use the following format:

Query: the input query you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of ['search_web']
Action Input: the input to the action
Observation: the result of the action
... (the Thought/Action/Observation can repeat any number of times)
Thought: I now know the final answer!
Final Answer: the answer to the original input query

[USER] 
Query: Find me some good tutorials on how to get started with Python.

[ASSISTANT] 
Thought: To find good tutorials on getting started with Python, I should search the web for beginner-friendly resources.

Action: search_web
Action Input: Python tutorials for beginners

[USER] 
Observation: ExceptionInfo(name='ConnectionError', message='No Internet connection.')

[ASSISTANT] 
Sorry, I was unable to retrieve any information due to a connection erro